In [ ]:
!pip install AugmentedSocialScientist, pandas

In [1]:
import os
# os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

from torch import cuda

cuda.get_device_name(0)

'Tesla T4'

In [2]:
import pandas as pd

annotations_file = pd.read_csv("../data/intermediate/current_annotation_file.csv")
annotations_file.sample(5)

,sentence_id,Personne,sentence_text
1437,1240920,NaN,Ce ne sont \npas non plus celles applicables a...
1033,1220629,NaN,"Le post humanisme, couplé au transhumanisme, v..."
1397,1120148,NaN,Et la création d’embryons \nin vitro à seule ...
105,680159,Genre/espèce humaine,"Dès lors, il ne faut surtout pas parler de ten..."
283,60156,Personne à protéger,Nous savons que le SIDA peut être mortel et qu...


In [3]:
annotations = annotations_file.dropna()
annotations.sample(5)

,sentence_id,Personne,sentence_text
534,1190207,Personne relationnelle,"En effet, il \nest essentiel qu’un autodépista..."
91,530014,Matériel biologique,et sont utilisées comme systèmes cellulaires p...
85,1370303,Genre/espèce humaine,Intégrer la santé publique aux biens communs (...
579,30148,Individu,Elle nous conduit à répondre à une série de qu...
574,820122,Humain comme empathie,C’est ce qu’on appelle le masque ou le triangl...


In [4]:
from sklearn.model_selection import train_test_split
 
train_data, test_data = train_test_split(annotations, train_size=0.67)

In [5]:
from AugmentedSocialScientist.models import Camembert

bert = Camembert()

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


There are 1 GPU(s) available.
We will use GPU 0: Tesla T4


In [6]:
train_loader = bert.encode(train_data.sentence_text.values,
train_data.Personne.values, batch_size=16)

  0%|          | 0/466 [00:00<?, ?it/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (1022 > 512). Running this sequence through the model will result in indexing errors


  0%|          | 0/466 [00:00<?, ?it/s]

label ids: {'Agent responsable': 0, 'Genre/espèce humaine': 1, 'Humain comme empathie': 2, 'Individu': 3, 'Matériel biologique': 4, 'Personne relationnelle': 5, 'Personne à protéger': 6, 'Poubelle': 7}


In [7]:
test_loader = bert.encode(test_data.sentence_text.values,
test_data.Personne.values, batch_size=16)

  0%|          | 0/231 [00:00<?, ?it/s]

  0%|          | 0/231 [00:00<?, ?it/s]

label ids: {'Agent responsable': 0, 'Genre/espèce humaine': 1, 'Humain comme empathie': 2, 'Individu': 3, 'Matériel biologique': 4, 'Personne relationnelle': 5, 'Personne à protéger': 6, 'Poubelle': 7}


In [8]:
scores = bert.run_training(
    train_loader,
    test_loader,
    n_epochs=2,
    lr=5e-5,
    random_state=1968,
    save_model_as='clickbait'
    )

Some weights of CamembertForSequenceClassification were not initialized from the model checkpoint at camembert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/opt/mamba/lib/python3.11/site-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(



======== Epoch 1 / 2 ========
Training...

  Average training loss: 1.96
  Training took: 0:00:40

Running Validation...


In [10]:
pred_data = annotations_file[annotations_file.isna().any(axis=1)].drop('Personne', axis=1)
pred_loader = bert.encode(pred_data.sentence_text.values, batch_size=16)
pred_proba = bert.predict_with_model(pred_loader, model_path='./models/clickbait')
pred_data['pred_label'] = np.argmax(pred_proba, axis=1)
pred_data['pred_proba'] = np.max(pred_proba, axis=1)